In [25]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from skimage.metrics import structural_similarity as ssim

In [26]:
# Function to calculate the third quartile (Q3) and first quartile (Q1)
def calculate_quartiles(edge_counts):
    q1 = np.percentile(edge_counts, 20)
    q3 = np.percentile(edge_counts, 90)
    return q1, q3

In [27]:
# Function to get the edge count of an image
def get_edge_count(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.countNonZero(edges)

In [5]:
# Function to save images based on edge count thresholds
def save_images(vidcap, q1_threshold, q3_threshold, output_dir_blurry, output_dir_under_q1):
    count_not_blurry = 0
    count_under_q1 = 0
    total_count = 0

    while vidcap.isOpened():
        success, image = vidcap.read()
        if not success:
            break
        
        total_count += 1
        edge_count = get_edge_count(image)

        # Save image if it's not blurry
        if edge_count >= q3_threshold:
            count_not_blurry += 1
            cv2.imwrite(f'{output_dir_blurry}/{count_not_blurry}.png', image)

        # Save image if edge count is under Q1
        if edge_count < q1_threshold:
            count_under_q1 += 1
            cv2.imwrite(f'{output_dir_under_q1}/{count_under_q1}.png', image)

    print(f"Total frames processed: {total_count}")
    print(f"Total frames not blurry: {count_not_blurry}")
    print(f"Total frames under Q1: {count_under_q1}")

In [28]:
print("Starting video processing")
#path of video
edge_counts = []
# read video from same directory in folder testing/testingBTA.mov
vidcap = cv2.VideoCapture('/Users/bachul/Downloads/IMG_7911.mov')
success, image = vidcap.read()

print("Video read successfully")

Starting video processing
Video read successfully


In [30]:
# Collect edge counts to calculate Q1 and Q3
counter = 0
while vidcap.isOpened():
    success, image = vidcap.read()
    if not success:
        break
    cv2.imwrite(f'./images/result/{counter}.png', image)
    counter += 1
    edge_count = get_edge_count(image)
    edge_counts.append(edge_count)

In [ ]:
# Print total count
total_frames = len(edge_counts)
print("Total frames: ", total_frames)

# Calculate Q1 and Q3
q1_threshold, q3_threshold = calculate_quartiles(edge_counts)
print(f"Edge Threshold (Q1): {q1_threshold}")
print(f"Edge Threshold (Q3): {q3_threshold}")

# Define output directories (make sure these directories exist)
output_dir_blurry = './testing/result/above90'
output_dir_under_q1 = './testing/result/under20'

# Reset the video capture and save images based on thresholds
vidcap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to start
save_images(vidcap, q1_threshold, q3_threshold, output_dir_blurry, output_dir_under_q1)
vidcap.release()

# Image Similarity Detection

## SSIM for frame comparison
Here’s a simple breakdown of how SSIM works:

1.	Human Perception Focus: SSIM models the way humans perceive images, emphasizing luminance (brightness), contrast, and structure (shapes and edges) to judge similarity. It goes beyond pixel-by-pixel comparison.
2.	Range of Values: SSIM gives a score between 0 and 1:
	2.1. 1 means the images are identical.
	2.2. 0 means the images are completely different.
3.	Usage in Frame Comparison: When comparing video frames, SSIM helps identify frames that are very similar (e.g., differences due to noise or minor changes). This is useful for keeping only distinct frames and discarding redundant ones.

In short, SSIM is a useful tool for filtering out nearly identical frames and improving efficiency in video processing tasks.

In [31]:
# Similarity frames function using SSIM
def filter_similar_frames(frames, threshold):
    """
    Filters out similar frames using the Structural Similarity Index (SSIM).

    Parameters:
        frames (list): List of frames (images) to be filtered.
        threshold (float): SSIM threshold for determining frame similarity.

    Returns:
        list: Filtered list of frames.
    """
    if not frames:
        return []

    filtered_frames = [frames[0]]  # Always keep the first frame
    for i in range(1, len(frames)):
        last_frame = filtered_frames[-1]
        current_frame = frames[i]
        
        # Convert frames to grayscale for SSIM comparison
        gray_last = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
        gray_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate SSIM between the two frames
        score, _ = ssim(gray_last, gray_current, full=True)
        print(f"SSIM Score: {score}")
        
        # If frames are not similar, keep the current frame
        if score < threshold:
            filtered_frames.append(current_frame)
    
    return filtered_frames

In [22]:
# Function to load all images from a directory
def load_images_from_folder(folder_path):
    frames = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # Adjust extensions as needed
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                frames.append(img)
    return frames

In [24]:
# Testing the similarity filter
# Load images
image_folder = './images/result/'

# Testing 2 images
pictA = cv2.imread(f'{image_folder}2.png')
pictB = cv2.imread(f'{image_folder}482.png')

# Create a list of frames
frames = [pictA, pictB]

filtered_frames = filter_similar_frames(frames, threshold=0.9)
print(f"Images after similarity filter: {len(filtered_frames)}")

# Display results with a timeout (e.g., 1 second per frame)
for i, frame in enumerate(filtered_frames):
    cv2.imshow(f'Filtered Frame {i+1}', frame)
    # Wait for 1 second (1000 milliseconds) or close if any key is pressed
    if cv2.waitKey(1000) & 0xFF == ord('q'):  # Press 'q' to quit early
        break

# Ensure all windows are closed properly
cv2.destroyAllWindows()
# END Testing 2 images

# # Load all images from the folder
# frames = load_images_from_folder(image_folder)

# # Call the function
# filtered_frames = filter_similar_frames(frames, threshold=0.9)
# print(f"Images after similarity filter: {len(filtered_frames)}")



SSIM Score: 0.92521315001758
SSIM Score: 0.9214995366896509
SSIM Score: 0.9238850414702077
SSIM Score: 0.9195843988692584
SSIM Score: 0.8648493137452807
SSIM Score: 0.925010953434446
SSIM Score: 0.9135423330473873
SSIM Score: 0.9309943588521156
SSIM Score: 0.9236824335788417
SSIM Score: 0.9248380804120038
SSIM Score: 0.9524074847392155
SSIM Score: 0.9296275902747414
SSIM Score: 0.9397643824608908
SSIM Score: 0.942423283772407
SSIM Score: 0.9327231148976287
SSIM Score: 0.9585087119776643
SSIM Score: 0.9225864697895472
SSIM Score: 0.9256463968963129
SSIM Score: 0.9258687860550197
SSIM Score: 0.9165394094324714
SSIM Score: 0.9855870771295228
SSIM Score: 0.925989970316468
SSIM Score: 0.8764968830617635
SSIM Score: 0.9291194921817727
SSIM Score: 0.9377546191667816
SSIM Score: 0.935000291935534
SSIM Score: 0.9255406599840796
SSIM Score: 0.918229318234147
SSIM Score: 0.9312030751482729
SSIM Score: 0.9510376529490402
SSIM Score: 0.9342361328114029
SSIM Score: 0.9343077459882838
SSIM Score: 0.9

# Model Validation for Classification Weather

Model has been trained on google colab :
https://colab.research.google.com/drive/1bAg4lleQg5IhuiprPHMZdj9kHj65giuE?usp=sharing 

In [38]:
trainedModel = YOLO('./machine-learning/model-frame-decent.pt')

dataTest = './images/bacteria.jpg'
resultTest = trainedModel(dataTest)

getNames = resultTest[0].names
getProbs = resultTest[0].probs.data.tolist()
getClassifications = getNames[np.argmax(getProbs)]
if getClassifications == "cloudy":
    print("The weather is cloudy")
else:
    print("The weather is not cloudy")

print(getNames)
print(getProbs)
print(getClassifications)


image 1/1 /Users/bachul/Documents/Projects/Macro/OculabImageStitchingPredictionBackend/images/bacteria.jpg: 64x64 not decent 0.81, decent 0.19, 1.4ms
Speed: 1.2ms preprocess, 1.4ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)
The weather is not cloudy
{0: 'decent', 1: 'not decent'}
[0.18621647357940674, 0.813783586025238]
not decent


: 